In [3]:
%%writefile vector_add.cu

#include <iostream>
#include <stdio.h> // Include for printf

__global__ void vectorMatrixMult(const float* A, const float* B, float* C, int N) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < N) {
      float sum=0.0f;
      for (int j = 0; j < N; j++) {
         sum += A[i*N+j]*B[j];
         // Add printf statements here to observe variable changes
         printf("Thread %d, i=%d, j=%d, A[i*N+j]=%.2f, B[j]=%.2f, sum=%.2f\n", threadIdx.x, i, j, A[i*N+j], B[j], sum);
      }
      C[i]=sum;
      printf("Thread %d, i=%d, final sum=%.2f\n", threadIdx.x, i, sum);
}}

int main() {
    //initialize the matrix
    const int N = 10;
    float *A, *B, *C;

    // initialize the input matrices
    A = (float *)malloc( N*N* sizeof(float));
    B = (float *)malloc(N*sizeof(float));
    C = (float *)malloc(N*sizeof(float));


    for (int i = 0; i < N; i++)
    {
        for (int j = 0; j < N; j++)
        {
            A[i * N + j] = 1.0f;
        }
        B[i] = 2.0f;
        C[i] = 0.0f;
    }

    float *d_a, *d_b,*d_c;
    cudaMalloc(&d_a,N*N*sizeof(float));
    cudaMalloc(&d_b,N*sizeof(float));
    cudaMalloc(&d_c,N*sizeof(float));
    cudaMemcpy(d_a,A,N*N*sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,B,N*sizeof(float),cudaMemcpyHostToDevice);
    int blocksize=256;
    int gridsize = (N + blocksize - 1) / blocksize;
    vectorMatrixMult<<<gridsize,blocksize>>>(d_a,d_b,d_c,N);

  cudaDeviceSynchronize();
cudaMemcpy(C,d_c,N*sizeof(float),cudaMemcpyDeviceToHost);

     printf("A:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {

            printf("%.2f ", A[i * N + j]); // Prints each element with 2 decimal precision
        }
        printf("\n"); // Adds a newline after each row
    }

    printf("C:\n");
    for (int i = 0; i < N; i++) {


            printf("%.2f ",C[i]); // Prints each element with 2 decimal precision

    }
printf("\n");
     printf("B:\n");
    for (int i = 0; i < N; i++) {


            printf("%.2f ", B[i ]); // Prints each element with 2 decimal precision

    }



    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

}

Overwriting vector_add.cu


In [4]:
# Compile with the specified architecture
!nvcc vector_add.cu -o vector_add -gencode arch=compute_75,code=sm_75

# Run the executable
!./vector_add

A:
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00 
C:
0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 
B:
2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 2.00 